In [1]:
import os

import numpy as np
import pandas as pd

from IPython.display import display, Markdown
from recommender import RecommendationEngine
from utils import *

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie_df = pd.DataFrame(
    [
        ["Julie", 4., 5., 2.],
        ["Sal", 1., 2., 5., 3.],
        ["Matthew", 4., 4., 2., 5.],
    ],
    columns=["Name", "2001: A Space Odyssey", "Fargo", "Sense and Sensibility", "Lord of the Rings"],
)

movie_df = movie_df.set_index("Name")
display(movie_df)

,2001: A Space Odyssey,Fargo,Sense and Sensibility,Lord of the Rings
Name,,,,
Julie,4.0,5.0,2.0,NaN
Sal,1.0,2.0,5.0,3.0
Matthew,4.0,4.0,2.0,5.0


In [5]:
# Julie's user index
user_index = 0

item_matrix = movie_df.values
imputed_matrix = movie_df.fillna(movie_df.mean(axis=0))

similarity_matrix = cosine_similarity(imputed_matrix)
display(similarity_matrix)
similarity_scores = list(enumerate(similarity_matrix[user_index]))

unrated_products = movie_df.iloc[user_index][movie_df.iloc[user_index].isna()].index
weights = [x[1] for x in similarity_scores]
display(weights)

inferred_ratings = (movie_df[unrated_products].T * weights).T
# inferred_ratings = inferred_ratings.iloc[[x[0] for x in similarity_scores if x[1] > 0.8]].mean()
inferred_ratings = inferred_ratings.iloc[[x[0] for x in similarity_scores]].mean()

display(round(inferred_ratings))

array([[1.        , 0.73808313, 0.98360656],
       [0.73808313, 1.        , 0.75858544],
       [0.98360656, 0.75858544, 1.        ]])

[1.0, 0.7380831312942567, 0.9836065573770493]

Lord of the Rings    4.0
dtype: float64

In [6]:
sim_df = pd.DataFrame(similarity_matrix, 
             columns=['Julie', 'Sal', 'Matthew'], 
             index=['Julie', 'Sal', 'Matthew'])
display(sim_df)

,Julie,Sal,Matthew
Julie,1.000000,0.738083,0.983607
Sal,0.738083,1.000000,0.758585
Matthew,0.983607,0.758585,1.000000


In [7]:
x = pd.DataFrame(sim_df[['Julie']].transpose()).drop('Julie', axis=1).reset_index(drop=True)
display(x)

y = movie_df[unrated_products].T.drop('Julie', axis=1)
display(y)

z = pd.DataFrame(x.values * y.values, columns=['Weighted Sal Score', 'Weighted Matthew Score'], index=['Lord of the Rings'])
display(z)

,Sal,Matthew
0,0.738083,0.983607


Name,Sal,Matthew
Lord of the Rings,3.0,5.0


,Weighted Sal Score,Weighted Matthew Score
Lord of the Rings,2.214249,4.918033


In [8]:
pd.DataFrame(inferred_ratings, columns=['Julie'])

,Julie
Lord of the Rings,3.566141
